# Ancestral Sequence Reconstruction 
Original code from Matt Spence, Mahakaran Sandhu, Josh Mitchell (Jackson Lab), modified by Matt Mortimer, matthew.mortimer@anu.edu.au, ORCID ID: https://orcid.org/0000-0002-8135-9319

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from Bio import AlignIO
from Bio import SeqIO
from log_func import log

In [ ]:
#Global variables that change with each project

project = 'SAL-AHL'
len_seq = 751 #Length of the codeml alignment (equal to the input alignment)
node_lst = []

in_rst = 'rst'
sub_dir = 'ASR'
f_path = 'ASR/rst'
codeml_file = 'Anc_seqs_codeml.fasta'
grasp_file = 'SAL-AHL_grasp_201028_joint-ancestors_GRASP.fasta'
ancNode_lst = [348,521,519,515,514,509,507,368,362,361,467,349]  # List of ancestral nodes for reconstruction


In [ ]:
def get_anc_trees(infile):
    """Retreives trees from CodeML RST file. Two trees will be returned, the first will include
    the ancestral node labels but no branchlengths, the second will include branch lengths but
    no node labels. Note that bootstraps are not stored in either tree and will have to be 
    retrieved from the in-tree file. Both trees have identical topologies, so branch lengths/
    node labels can be exchanged between them using tree manipulation packages. Credit to Matt S"""
    trees = []
    with open(infile) as codeML:
        for line in codeML:
            if line.startswith('(') and line.endswith(';\n'):
                trees.append(line.strip())    
    anc_nwk_bl = trees[0]
    anc_nwk_lab = trees[2]
    return(anc_nwk_lab, anc_nwk_bl)

def read_rst(infile, node_point, seq_length):
    """Reads to CodeML RST file and returns the posterior probability of a SINGLE node (designated by node_point).
    Note that read_rst returns a tuple with (node_point, node data), node data includes ALL raw sequence info
    from the RST file, not the extracted extracted float values corresponding to the posterior distribution.
    
    read_rst is intended to be run on a SINGLE node at a time and will not take a list of nodes to read. Instead,
    construct a for-loop of nodes and call read_rst once for each node. Credit to Matt S and Mahakaran"""
    
    data = open(infile, 'r')
    datas = data.readlines()
    cat_data = []
    for i in range(len(datas)):
        node_data = []
        if 'Prob distribution at node' in datas[i]:
            if node_point > 999:                      # Why is this '999'?
                if ',' in datas[i][26:30]:
                    continue
                else:
                    if int(datas[i][26:30]) == node_point:
                        node_data.append(datas[i+4:i+4+int(seq_length)])
                        node = datas[i][26:30]
                        node_data = node_data[0]
                        nodes = (node, node_data)
                        cat_data.append(nodes)

            else:
                if int(datas[i][26:29]) == node_point:
                    node_data.append(datas[i+4:i+4+int(seq_length)])
                    node = datas[i][26:29]
                    node_data = node_data[0]
                    nodes = (node, node_data)
                    cat_data.append(nodes)
    return cat_data


def pd_post_dist(rst):    
    """TAKES THE OUTPUT OF read_rst command. Retrieves the posterior distribution of a node, and returns
    it as a pandas dataframe of floating values indexed by site of the sequence. Note that pd_post_dist will
    not return an ML sequence, only the numerical values of the full posterior distribution.
    
    pd_post_dist is intended to be run on a SINGLE output from read_rst and will not parse a list of rst values.
    Instead, construct a for loop with read_rst and pd_post_dist. Credit to Matt S"""
    for i in rst:
        data = i[1]
    aa_site_list = []
    aa_post_dict = []
    for site, i in enumerate(data):
        z = []
        for m, n in enumerate(i.split(':')):
            if m == 1:
                z.append(n.strip())
                d = {site+1:z}
                aa_post_dict.append(d)
    for ind, dist in enumerate(aa_post_dict):
        if ind == 0:
            df = pd.DataFrame.from_dict(dist).transpose()
        else:
            df = pd.concat([df, pd.DataFrame.from_dict(dist).transpose()], axis = 0, sort = False) 
    df = df[0].str.split(' ', n = 19, expand = True)
    for col in range(20):    
        test_list = df[col].to_list()
        z = []
        for i in test_list:
            z.append(re.search('\(([^)]+)', i).group(1))
        df[col] = z
    return df

def pd_ML_residues(infile, nodes, seq_length): 
    """DOES NOT TAKE THE OUTPUT OF read_rst or pd_post_dist. pd_ML_residues calls read_rst, pd_post_dist commands to 
    retrieve the ML probabilities for each sequence in a list of nodes and returns them as rows of a pd dataframe
    indexed by node. Note that this will not return the ML sequence, only the ML probabilities at each site of each
    node.
    
    pd_ML_residues is intended to be run on a LIST of MULTIPLE node numbers, to allow easy manipulation of sites 
    that contribute to the single mean posterior probability. Credit to Matt S"""
    for i, node in enumerate(nodes):
        rst = read_rst(infile, node_point=int(node), seq_length=len_seq)
        df = pd_post_dist(rst)
        df['ML'] = df.max(axis =1)
        ML_prob = []
        ML_prob = df['ML'].to_list()
        if i == 0:
            ml_df = pd.DataFrame(ML_prob).transpose()
        else:
            ml_df = pd.concat([ml_df, pd.DataFrame(ML_prob).transpose()], axis = 0)
    ml_df.columns = list(range(1, seq_length+1))
    ml_df.index = nodes
    return ml_df


def branch_mutations(infile, no_branches):
    """Reads the codeML RST file and returns a dictionary of mutations (including posteriors)
    per branch and a pandas dataframe with branch and '/' separated list of mutations (not
    including posteriors). No_branches is the number of branches, which is equal to the 
    number of tips - 2. 
    
    branch_mutations() will retrieve mutations from all branches in the tree. For individual
    branches, use the output dictionary with the branch number. Branches with no mutations
    are replaced by np.nan. Credit to Matt S and Josh."""
    with open(infile, 'r') as file:
        branches_out = {}
        current_branch = None
        for line in file.readlines():
            if line.startswith("Branch "):
                current_branch = int(line.split()[1][:-1])
                branch_list = branches_out.setdefault(current_branch, [])
            elif line.strip() == "List of extant and reconstructed sequences":
                break
            elif line.strip() and current_branch:
                branch_list.append(line.split())
    new_dict = {}
    for key, value in branches_out.items():
        new_dict[key] = pd.DataFrame(value) 
    branch = []
    muts_list = []       
    for x in range(1, (no_branches+1)):
        z = []
        df = new_dict[x]
        if df.empty == True:
            continue
        else:
            df = new_dict[x]
            df[4] = df[4].replace({'-':np.nan})
            df = df.dropna()
            pos = df[0].tolist()
            ori = df[1].tolist()
            mut = df[4].tolist()
            for p,o,m in zip(pos, ori, mut):
                comb = str(o)+str(p)+str(m)
                z.append(comb) 
                y = '/'.join(z)
            branch.append(x)
            muts_list.append(y)
    branch_muts_df = pd.DataFrame(branch)
    branch_muts_df[1] = muts_list
    branch_muts_df.columns = ['branch', 'mutations']
    return new_dict, branch_muts_df

In [ ]:
def asr_processing(len_seq = len_seq, project = project, sub_dir = ''):
    '''
    Takes the following args
    'project' = project name as str
    'sub_dir' = subdir name of where rst file is location as str
    'len_seq' = the length of the alignment used for the ASR as int
    1) Retreives the two ancestral trees from the codeml 'rst' file and writes them to file as newick
    2) Takes the list of ancestral nodes of interest generates heatmaps of site by Amino acid, then 
        saves them to file
    Outputs:
        {project}_anc_lab.nwk
        {project}_anc_bl.nwk
        {anc}_HTH_PP.pdf where anc is each ancestral node in ancNode_lst
        Anc_seqs_codeml.fasta with all the ancestral sequences from the rst file
        ML_seq_df a dataframe of the ML probability/site for each node in the tree
    '''

    #Retreives the two anc trees from the RST and writing them to file

#     anc_lab, anc_bl = get_anc_trees(f'{f_path}')
#     print(anc_lab, file = open(f'{f_path}_anc_lab.nwk', 'a'))
#     print(anc_bl, file = open(f'{f_path}_anc_bl.nwk', 'a'))
    
    
#     for n in ancNode_lst:
#         anc = str(n)
#         anc_temp = read_rst(f'{f_path}',n, len_seq) #seq_length (3rd variable) is this just the alignment length?
#         anc_temp_pd = pd_post_dist(anc_temp)
#         anc_temp_pd.head(100)
#         anc_temp_pd = anc_temp_pd.astype(float)
#         aa = ['A','R','N','D','C','Q','E','G','H','I','L','K','M','F','P','S','T','W','Y','V']
#         plt.pcolor(anc_temp_pd, cmap='viridis')
#         plt.xlabel('Amino acid', fontsize = 10) 
#         plt.xticks(np.arange(0.5, 20.5, step=1), aa)
#         plt.ylabel('Site', fontsize = 10)
#         plt.title(f'{anc} posterior distribution', fontsize = 14)
#         plt.savefig(f'Anc{anc}_PP.pdf')
#         log(f'Anc{anc}_PP.pdf was created for {project} ASR')
    
#     # Converting codeml seqs to fasta 
#     ofile = open(f'ASR/Anc_seqs_codeml.fasta', 'a')

#     with open(f'{f_path}') as rst:
#         count = 0
#         for i in rst:
#             if i.startswith('node #'):
#                 n_id = i[6:10]
#                 n_id = n_id.strip()
#                 node_lst.append(int(n_id))
#                 seq = i[10:]
#                 seq = ''.join(seq.split())
#                 ofile.write('>Anc_' + n_id + '\n')
#                 ofile.write(seq + '\n')
#                 count += 1
#     log(f'Anc_seqs_codeml.fasta was created with {count} ancestral sequences')
#     ofile.close()
    
#     aln_codeml = AlignIO.read(f'ASR/{codeml_file}', 'fasta')
#     aln_grasp = AlignIO.read(f'ASR/{grasp_file}', 'fasta')

#     seq_ids = []
#     seqs = []
#     for i in range(len(aln_grasp)):
#         test_grasp = aln_grasp[i]
#         test_codeml = aln_codeml[i]
#         seq_ids.append(aln_codeml[i].id)
#         partial_seq = ''
#         for n,(g,c) in enumerate(zip(test_grasp,test_codeml)):
#             if g == '-':
#                 partial_seq+='-'
#             else:
#                 partial_seq+=c
#         seqs.append(partial_seq)
    
#     c_count = 0
#     for i,s in zip(seq_ids,seqs):
#         print('>'+i, file=open(f'ASR/Anc_seqs_corrected.txt', 'a'))
#         print(s, file=open(f'ASR/Anc_seqs_corrected.txt', 'a'))
#         c_count += 1
#     log(f'Anc_seqs_corrected.txt was created with {c_count} ancestral sequences')

    pp_dists = []
    aln_codeml = AlignIO.read(f'ASR/Anc_seqs_corrected.txt', 'fasta')
    anc_nodes = node_lst 
    for i in anc_nodes:
        pp_rst = read_rst(infile= f_path, node_point=int(i), seq_length=len_seq)
        pp_dist = pd_post_dist(pp_rst)
        pp_dists.append(pp_dist)

    all_sites = []
    for seq in aln_codeml:
        sites_to_drop = []
        for n,aa in enumerate(seq.seq):
            if aa == '-':
                sites_to_drop.append(n+1)
            else:
                continue
        all_sites.append(sites_to_drop)

    corrected_pp_dists = []
    ML_pp = []
    for i,n in zip(all_sites,pp_dists):
            pp_dist = n.drop(i, axis = 0)
            pp_dist['ML'] = pp_dist.max(axis =1)
            ML_prob = pp_dist['ML'].to_list()
            ML_pp.append(ML_prob)
            corrected_pp_dists.append(pp_dist)
    
    global mean_pp
    mean_pp = []
    for i in ML_pp:
        av = sum(i)/len(i)
        mean_pp.append(av)

#     for i,n in zip(aln_codeml,mean_pp):
#         print(i.id+'\t'+str(n), file=open(f'ASR/Anc_mean_pp.txt', 'a')) 
#     log(f'Anc_mean_pp.txt was created with the mean posterior probablities for each seqeunce in Anc_seqs_corrected.txt')
    
    
#     anc_tree,ex_tree = get_anc_trees(f_path)
#     print(anc_tree, file = open(f'ASR/Anc_tree.txt.treefile', 'a'))
#     log(f'Anc_tree.txt.treefile phylogentic tree was created in newick format')

# Runs the ASR processing
Merges codeml and grasp alignments

In [ ]:
asr_processing()

Two charts with posterior probabilities of the codeml and then the corrected ancestors

In [ ]:
with open(f'{f_path}') as rst:
    ML_seq_df = pd_ML_residues(infile=f_path, nodes=node_lst, seq_length=len_seq)

In [ ]:
new_col = list(range(1,751))
new_col.append('MPP')
ML_seq_df.columns = new_col


cm = plt.cm.get_cmap('viridis')
fig,ax = plt.subplots()

# Plot histogram.
n, bins, patches = plt.hist(ML_seq_df['MPP'], bins=20, edgecolor = 'black', linewidth = 1, zorder = 100)
bin_centers = 0.5 * (bins[:-1] + bins[1:])

# scale values to interval [0,1]
col = bin_centers - min(bin_centers)
col /= max(col)

for c, p in zip(col, patches):
    plt.setp(p, 'facecolor', cm(c))
    
plt.xlabel("Mean posterior probability", fontsize=8)  
plt.ylabel("Number of nodes", fontsize=8)
plt.xticks(fontsize=8)  
plt.yticks(fontsize=8)
plt.tick_params(axis='x', labelsize=8)
plt.tick_params(axis='y', labelsize=8)

ax.axvline(0.9, linewidth =1,alpha=1, color='gray', linestyle = 'dashed')

fig.set_size_inches(3.05,1.95)

plt.tight_layout()
plt.savefig('codeml_mean_pp.pdf', )

In [ ]:
cm = plt.cm.get_cmap('viridis')

fig,ax = plt.subplots()

# Plot histogram.
n, bins, patches = plt.hist(mean_pp, bins=20, edgecolor = 'black', linewidth = 1, zorder = 100)
bin_centers = 0.5 * (bins[:-1] + bins[1:])

# scale values to interval [0,1]
col = bin_centers - min(bin_centers)
col /= max(col)

for c, p in zip(col, patches):
    plt.setp(p, 'facecolor', cm(c))

plt.xlabel("Mean posterior probability", fontsize=8)  
plt.ylabel("Number of nodes", fontsize=8)
plt.xticks(fontsize=8)  
plt.yticks(fontsize=8)
plt.tick_params(axis='x', labelsize=8)
plt.tick_params(axis='y', labelsize=8)

ax.axvline(0.9, linewidth =1,alpha=1, color='gray', linestyle = 'dashed')

fig.set_size_inches(3.05,1.95)

plt.tight_layout()
plt.savefig('corrected_mean_pp.pdf', )

In [ ]:
mean_pp

Make fasta files of each ancestral node from the txt file 'Anc_tips.txt' from R (packages caper and ape), uses biopython to generate consensus sequences and then prints to file (Anc_consensus_equiv.fasta)

I need to add Anc_consensus_equiv.fasta to the ancestors and extant sequences file then generate the corelation matrix

In [ ]:
# Credit Matt Mortimer, work in progress - make into a 

from Bio import AlignIO
from Bio.Align import AlignInfo
from Bio import SeqIO
import os

files = os.listdir()

master_dict = {}

# 'ASR_lst.fasta' is the alignment that the ancestral sequences were made from 
# Generates a dictionary of Unids and Sequences called 'master_dict'
with open(f'ASR_lst.fasta', 'r') as fasta:
        k = ""
        v = ""
        for line in fasta:
            if line.startswith('>'):
                k = line.rstrip().lstrip(">")
            else:
                v = line.strip('\n')
            master_dict.update({k: v})    

# 'Anc_tips.txt' is a file of all decendants/tips from a given ancestor/node generated in R
# Takes the ancestor number from the file and converts to a str to use in the file name
with open('Anc_tips.txt', 'r') as tips:
    for i in tips:
        dec = []
        if i.startswith("Ancestor:  "):
            anc = i[-5:]
            anc = anc.strip()
            anc = str(anc)
        else:
            # All tip numbers / unids are added as elements to list 'dec'
            dec.append(i.strip())
        
        # Creates/appends to a fasta file sequences taken from 'master_dict' if in the list 'dec'
        with open(f'Anc_{anc}_decendents.fasta', 'a') as fasta:
            w_dict = {k: v for k, v in master_dictt.items() if k in dec} 
            for k,v in w_dict.items():
                fasta.write('>' + k + '\n')
                fasta.write(v + '\n')

# Uses BioPython to make concensus sequences from the 'Anc_{anc}_decendents.fasta' generated above          
for f in files:
    if f.startswith("Anc_tip"):
        pass
    # If the file is not an 'Anc_tip' file it will proceed to below. Can't have anything else in the dir other
    # than 'Anc_tip' or 'Anc_{anc}_decendents.fasta' files or it will fail
    elif f.startswith("Anc_"):
        # Take the name of the Ancestor the sequences are decendants of and save as variable 'anc' 
        anc = f[:7]
        # Uses BioPython / AlignIO to parse the fasta file then 'summerises' it assigning the object as variable 'summary_align'
        aln = AlignIO.read(f, 'fasta')
        summary_align = AlignInfo.SummaryInfo(aln)
        # Generates a concensus sequence, assigns it to variable 'consensus'
        consensus = summary_align.gap_consensus(threshold=0, ambiguous='X', consensus_alpha=None, require_multiple=0)
        print(str(consensus))
        # Creates or appends file 'Anc_consensus_equiv.fasta'. Each concesus sequence has a fasta header labelled by the ancestor equiv
        with open(f'Anc_consensus_equiv.fasta', 'a') as con:
            con.write(f">{anc}_consensus\n")
            con.write(str(consensus) + "\n")
    else:
        pass